# Tarea 03

Resuelve los siguientes problemas por medio de programación funcional

**1.** Dada `list1`, reemplaza cada elemento `a` por `--`

```python
list1 = ["a", "b", "a", "d", "a", "a", "c"]
```

**2.** Obten el número de carácteres para cada elemento de la lista `acciones`.

```python
acciones = ['BIMBO', 'CEMEX', 'IENOVA', 'KOF', 'PE&OLES']
```

**3.** Dada la lista `nums`, suma a cada entrada un `1`.

```python
nums = ["1", "3", "5", "7", "11", "13"]
```

**4.** Dada la lista `numbers`, crea un diccionario dónde la llave sea la entrada de cada lista y el valor la longitud de cada entrada  
```python
numbers = ["one", "two", "three", "four", "five", "six"]
```

**5.** Crear un dictionary comprehension de python donde las llaves sean los números 1 al $n$ y, los valores, una lista con sus potencias del $2$ al $m$

Por ejemplo, si $n=6$, y $m=5$ entonces el programa arrojaría. Considera variables `n` y `m` para generar el programa.
```python
{1: [1, 1, 1, 1],
 2: [4, 8, 16, 32],
 3: [9, 27, 81, 243],
 4: [16, 64, 256, 1024],
 5: [25, 125, 625, 3125],
 6: [36, 216, 1296, 7776]}
```

**6.** Crea un list comprehension que arroje el siguiente resultado:

```python
[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
 [3, 6, 9, 12, 15, 18, 21, 24, 27, 30],
 [4, 8, 12, 16, 20, 24, 28, 32, 36, 40],
 [5, 10, 15, 20, 25, 30, 35, 40, 45, 50],
 [6, 12, 18, 24, 30, 36, 42, 48, 54, 60],
 [7, 14, 21, 28, 35, 42, 49, 56, 63, 70],
 [8, 16, 24, 32, 40, 48, 56, 64, 72, 80],
 [9, 18, 27, 36, 45, 54, 63, 72, 81, 90],
 [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]]
```

**7.** 
Considerando el universo `C = ['Nokia', 'Citigroup', 'Union Pacific', 'Jp Morgan Chase', 'Morgan Stanley', 'Lloyds Tsb', 'Wells Fargo', 'Ford Motor', 'Pfizer']` y considerando $F(c) \equiv c \text{ contiene la letra 'o'}$, escribe una expresión en python para la siguiente fbf.

$$\forall c \in C. F(c)$$

**8.** Considerando el universo $U = \mathbb{Z}^+$, el subcojunto $A = \{1, 3, 6, 7, 11\} \subseteq U$ y el predicado 
$$
    F(x) \equiv x \text{ es número primo}
$$

Escribe en python una expresión que value la siguiente fbf:
$$
    \forall a \in A. F(a)
$$

*Hint: considera la siguiente función para validar si `a` es un número primo*
```python
def is_prime(x):
    """
    Parameters
    ----------
    x: int
        positive integer greater than or equal to two
    
    Returns
    -------
    bool: whether x is prime or not
    """
    for i in range(2, x):
        if x % i == 0:
            return False
    return True
```

**9.** Considerando el universo $U = \mathbb{Z}^+$, los subconjuntos $A = \{2, 3, 5, 7, 9\}$; $B = \{1, 3, 7, 15, 31, 63, 127, 255, 511, 1023\}$, escribe una fbf para el siguiente enunciado:
> todo elemento $a \in A$ tiene al menos un elemento $b \in B$ tal que si $2 ^ a  - 1 = b$, entonces $a, b$ son números primos.

Evalua tu expresión en python

$F(a, b) \equiv 2^a - 1 = b \rightarrow a, b \in Primes$

$\forall a\in A.\exists b \in B. F(a, b)$

**10.** Se definió la función `pyramid` que tome un valor `v` e imprima una pirámide de la siguiente manera

```python
>>> pyramid(4)
   1
  2 2
 3 3 3
4 4 4 4
```

La pirámide deberá tomar forma dependiendo `v` a fin de no perder la estructura. Por ejemplo, si `v = 11`
```python
>>> pyramid(10)
               01                
              02 02              
            03 03 03             
           04 04 04 04           
         05 05 05 05 05          
        06 06 06 06 06 06        
      07 07 07 07 07 07 07       
     08 08 08 08 08 08 08 08     
   09 09 09 09 09 09 09 09 09    
  10 10 10 10 10 10 10 10 10 10  
11 11 11 11 11 11 11 11 11 11 11 
```
En el caso donde `v >= 100`, las unidades deberían tener dos ceros de *padding* y así sucesivamente.

Se tiene la siguiente función que trata de resolver este problema
```python
def pyramid(v):
vlen = len(str(v))
    max_len = v * 2
    for i in range(v + 1):
        label = format(i, "02")
    print((f"{label} " * i).center(v))
```

Usando `pbd` y leyendo los errores, soluciona el programa. 